# Neural nets as Sudoku solvers

Since the dataset was initially created for neural-nets, I focus on this approach. Obviously, there are other ways to solve Sudoku puzzle, some of them are way more intuitive than neural-nets. However, it was an opportunity to get a first hands-on with Keras framework !

I read the https://github.com/Kyubyong/sudoku to see if CNNs can crack Sudoku puzzles. 
I was wondering if we could achieve similar results with a simpler neural net architecture.

I decided to dig more in the direction of the following tip Kyubyong mentionned about predicting :
> I use a simple trick in inference. Instead of cracking the whole blanks all at once, I fill in a single blank where the prediction is the most probable among the all predictions.

I used the same trick but in **both prediction phase AND training phase.**
One of my workmate came up with this idea. Thanks to him !

In a nutshell : 
* **step 1 - Calibration**: train network to reproduce a correct grid identically.
* **step 2 - Pull off** : for each training quizz, pull off one digit, and train to recover it.
* **step 3 - pull off one more**: pull off 2 digits (the first one may be different than *step 2*).
* **...**
* **penultimate step**: pull off a reasonible number off digits to build a realistic Sudoku grid (arround 50)
* **final step**: Predict smartly. Take a quizz, get predictions, place the digit which the network is more confident with to complete one single blank. Then feed the new quizz (with one blank less) into the network again. And so on until there is no more blank.

It works pretty well, with a final **99.7 % accuracy**. Feel free to fine-tune the model and share your results :).

Merry end of the year !

Dithyrambe

*NOTE : Training the network is quite long, you'll have to run the notebook on your environment since Kaggle kills it after 3600 sec*

In [1]:
# imports
import numpy as np
import pandas as pd

from keras import Model, Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Flatten, Input
from keras.utils import to_categorical

Using TensorFlow backend.


Define some useful functions. Docstrings should be enough for explanations. The whole trick is in the `batch_smart_solve` function.

In [2]:
def load_data(nb_train=40000, nb_test=10000, full=False):
    """
    Function to load data in the keras way.
    
    Parameters
    ----------
    nb_train (int): Number of training examples
    nb_test (int): Number of testing examples
    full (bool): If True, whole csv will be loaded, nb_test will be ignored
    
    Returns
    -------
    Xtrain, ytrain (np.array, np.array),
        shapes (nb_train, 9, 9), (nb_train, 9, 9): Training samples
    Xtest, ytest (np.array, np.array), 
        shapes (nb_test, 9, 9), (nb_test, 9, 9): Testing samples
    """
    # if full is true, load the whole dataset
    if full:
        sudokus = pd.read_csv('./datasets/sudoku-kaggle.csv').values
    else:
        sudokus = next(
            pd.read_csv('./datasets/sudoku-kaggle.csv', chunksize=(nb_train + nb_test))
        ).values
        
    quizzes, solutions = sudokus.T
    flatX = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9))
                      for flatten_grid in quizzes])
    flaty = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9))
                      for flatten_grid in solutions])
    
    return (flatX[:nb_train], flaty[:nb_train]), (flatX[nb_train:], flaty[nb_train:])


def diff(grids_true, grids_pred):
    """
    This function shows how well predicted quizzes fit to actual solutions.
    It will store sum of differences for each pair (solution, guess)
    
    Parameters
    ----------
    grids_true (np.array), shape (?, 9, 9): Real solutions to guess in the digit repesentation
    grids_pred (np.array), shape (?, 9, 9): Guesses
    
    Returns
    -------
    diff (np.array), shape (?,): Number of differences for each pair (solution, guess)
    """
    return (grids_true != grids_pred).sum((1, 2))


def delete_digits(X, n_delet=1):
    """
    This function is used to create sudoku quizzes from solutions
    
    Parameters
    ----------
    X (np.array), shape (?, 9, 9, 9|10): input solutions grids.
    n_delet (integer): max number of digit to suppress from original solutions
    
    Returns
    -------
    grids: np.array of grids to guess in one-hot way. Shape: (?, 9, 9, 10)
    """
    grids = X.argmax(3)  # get the grid in a (9, 9) integer shape
    for grid in grids:
        grid.flat[np.random.randint(0, 81, n_delet)] = 0  # generate blanks (replace = True)
        
    return to_categorical(grids)


def batch_smart_solve(grids, solver):
    """
    NOTE : This function is ugly, feel free to optimize the code ...
    
    This function solves quizzes in the "smart" way. 
    It will fill blanks one after the other. Each time a digit is filled, 
    the new grid will be fed again to the solver to predict the next digit. 
    Again and again, until there is no more blank
    
    Parameters
    ----------
    grids (np.array), shape (?, 9, 9): Batch of quizzes to solve (smartly ;))
    solver (keras.model): The neural net solver
    
    Returns
    -------
    grids (np.array), shape (?, 9, 9): Smartly solved quizzes.
    """
    grids = grids.copy()
    for _ in range((grids == 0).sum((1, 2)).max()):
        preds = np.array(solver.predict(to_categorical(grids)))  # get predictions
        probs = preds.max(2).T  # get highest probability for each 81 digit to predict
        values = preds.argmax(2).T + 1  # get corresponding values
        zeros = (grids == 0).reshape((grids.shape[0], 81))  # get blank positions

        for grid, prob, value, zero in zip(grids, probs, values, zeros):
            if any(zero):  # don't try to fill already completed grid
                where = np.where(zero)[0]  # focus on blanks only
                confidence_position = where[prob[zero].argmax()]  # best score FOR A ZERO VALUE (confident blank)
                confidence_value = value[confidence_position]  # get corresponding value
                grid.flat[confidence_position] = confidence_value  # fill digit inplace
    return grids

### Prepare data for the training phase.

Since we built a `delete_digits` function, we will create our own training set from `ytrain` directly.
Also note that we don't want targets to be (9, 9, 10) shaped. Since predicting zeros is useless ... we want to FILL zeros

In [3]:
input_shape = (9, 9, 10)
(_, ytrain), (Xtest, ytest) = load_data()  # We won't use _. We will work directly with ytrain

# one-hot-encoding --> shapes become :
# (?, 9, 9, 10) for Xs
# (?, 9, 9, 9) for ys
Xtrain = to_categorical(ytrain).astype('float32')  # from ytrain cause we will creates quizzes from solusions
Xtest = to_categorical(Xtest).astype('float32')

ytrain = to_categorical(ytrain-1).astype('float32') # (y - 1) because we 
ytest = to_categorical(ytest-1).astype('float32')   # don't want to predict zeros

### Lets now define the keras model
It consists in a simple 2 `Dense` layers and 81 `Dense` output layer (one for each digits to predict).

In [4]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=input_shape))
model.add(Dropout(rate=0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(Flatten())

grid = Input(shape=input_shape)  # inputs
features = model(grid)  # commons features

# define one Dense layer for each of the digit we want to predict
digit_placeholders = [
    Dense(9, activation='softmax')(features)
    for i in range(81)
]

solver = Model(grid, digit_placeholders)  # build the whole model
solver.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Train model to output solution from a solution.
This is part of the trick. The idea comes from a workmate. Thanks to him.
The idea is to make the network outputs the solution for the easiest quizz. That is to say ... the solution itself

***solver(solution) &rarr; solution***

This will calibrate weights to reproduce the grid (as auto-encoders do ?).

In [5]:
solver.fit(
    delete_digits(Xtrain, 0),  # we don't delete any digit for now
    [ytrain[:, i, j, :] for i in range(9) for j in range(9)],  # each digit of solution
    batch_size=128,
    epochs=1,  # 1 epoch should be enough for the task
    verbose=1,
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
40000/40000 [==============================] - 89s 2ms/step - loss: 41.9666 - dense_3_loss: 0.5187 - dense_4_loss: 0.5144 - dense_5_loss: 0.5216 - dense_6_loss: 0.5206 - dense_7_loss: 0.5210 - dense_8_loss: 0.5180 - dense_9_loss: 0.5176 - dense_10_loss: 0.5188 - dense_11_loss: 0.5132 - dense_12_loss: 0.5259 - dense_13_loss: 0.5173 - dense_14_loss: 0.5157 - dense_15_loss: 0.5151 - dense_16_loss: 0.5243 - dense_17_loss: 0.5160 - dense_18_loss: 0.5194 - dense_19_loss: 0.5170 - dense_20_loss: 0.5227 - dense_21_loss: 0.5153 - dense_22_loss: 0.5212 - dense_23_loss: 0.5094 - dense_24_loss: 0.5111 - dense_25_loss: 0.5203 - dense_26_loss: 0.5192 - dense_27_loss: 0.5193 - dense_28_loss: 0.5201 - dense_29_loss: 0.5179 - dense_30_loss: 0.5217 - dense_31_loss: 0.5177 - dense_32_loss: 0.5196 - dense_33_loss: 0.5208 - dense_34_loss: 0.5215 - dense_35_loss: 0.5219 - dense_36_loss: 0.5085 - dense_37_loss: 0.5165 - dense_38_loss: 0.5168 - dense_3

### Train model to guess something harder ... and harder ... and harder. One drop at a time.
Here we will track validation loss to avoid over fitting

First, we will pull off *a single* digit from the solution. This is quite a simple quizz to guess.
We will train the network until it overfits

Next, lets pull off 2 of them, ... then 3, ... then 4, and so on since we reach hard enough grid with around 55 missing digits.

In [6]:
early_stop = EarlyStopping(patience=2, verbose=1)

i = 1
for nb_epochs, nb_delete in zip(
        [1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],  # epochs for each round
        [1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55]  # digit to pull off
):
    print('Pass n° {} ...'.format(i))
    i += 1
    
    solver.fit(
        delete_digits(Xtrain, nb_delete),  # delete digits from training sample
        [ytrain[:, i, j, :] for i in range(9) for j in range(9)],
        validation_data=(
            delete_digits(Xtrain, nb_delete), # delete same amount of digit from validation sample
            [ytrain[:, i, j, :] for i in range(9) for j in range(9)]),
        batch_size=128,
        epochs=nb_epochs,
        verbose=1,
        callbacks=[early_stop]
    )

Pass n° 1 ...
Train on 40000 samples, validate on 40000 samples
Epoch 1/1
40000/40000 [==============================] - 87s 2ms/step - loss: 1.5908 - dense_3_loss: 0.0215 - dense_4_loss: 0.0178 - dense_5_loss: 0.0212 - dense_6_loss: 0.0199 - dense_7_loss: 0.0210 - dense_8_loss: 0.0192 - dense_9_loss: 0.0206 - dense_10_loss: 0.0200 - dense_11_loss: 0.0169 - dense_12_loss: 0.0198 - dense_13_loss: 0.0198 - dense_14_loss: 0.0201 - dense_15_loss: 0.0196 - dense_16_loss: 0.0189 - dense_17_loss: 0.0172 - dense_18_loss: 0.0189 - dense_19_loss: 0.0194 - dense_20_loss: 0.0196 - dense_21_loss: 0.0193 - dense_22_loss: 0.0212 - dense_23_loss: 0.0210 - dense_24_loss: 0.0180 - dense_25_loss: 0.0195 - dense_26_loss: 0.0199 - dense_27_loss: 0.0182 - dense_28_loss: 0.0191 - dense_29_loss: 0.0206 - dense_30_loss: 0.0190 - dense_31_loss: 0.0199 - dense_32_loss: 0.0211 - dense_33_loss: 0.0180 - dense_34_loss: 0.0190 - dense_35_loss: 0.0209 - dense_36_loss: 0.0184 - dense_37_loss: 0.0182 - dense_38_loss: 0

Pass n° 2 ...
Train on 40000 samples, validate on 40000 samples
Epoch 1/2
40000/40000 [==============================] - 85s 2ms/step - loss: 1.9857 - dense_3_loss: 0.0239 - dense_4_loss: 0.0227 - dense_5_loss: 0.0254 - dense_6_loss: 0.0232 - dense_7_loss: 0.0229 - dense_8_loss: 0.0241 - dense_9_loss: 0.0263 - dense_10_loss: 0.0220 - dense_11_loss: 0.0254 - dense_12_loss: 0.0230 - dense_13_loss: 0.0248 - dense_14_loss: 0.0224 - dense_15_loss: 0.0243 - dense_16_loss: 0.0230 - dense_17_loss: 0.0252 - dense_18_loss: 0.0229 - dense_19_loss: 0.0245 - dense_20_loss: 0.0243 - dense_21_loss: 0.0244 - dense_22_loss: 0.0241 - dense_23_loss: 0.0261 - dense_24_loss: 0.0212 - dense_25_loss: 0.0231 - dense_26_loss: 0.0215 - dense_27_loss: 0.0262 - dense_28_loss: 0.0236 - dense_29_loss: 0.0249 - dense_30_loss: 0.0218 - dense_31_loss: 0.0265 - dense_32_loss: 0.0252 - dense_33_loss: 0.0268 - dense_34_loss: 0.0249 - dense_35_loss: 0.0242 - dense_36_loss: 0.0237 - dense_37_loss: 0.0255 - dense_38_loss: 0

Epoch 2/2
40000/40000 [==============================] - 85s 2ms/step - loss: 1.5485 - dense_3_loss: 0.0189 - dense_4_loss: 0.0190 - dense_5_loss: 0.0192 - dense_6_loss: 0.0217 - dense_7_loss: 0.0191 - dense_8_loss: 0.0199 - dense_9_loss: 0.0146 - dense_10_loss: 0.0188 - dense_11_loss: 0.0182 - dense_12_loss: 0.0199 - dense_13_loss: 0.0186 - dense_14_loss: 0.0187 - dense_15_loss: 0.0209 - dense_16_loss: 0.0194 - dense_17_loss: 0.0188 - dense_18_loss: 0.0176 - dense_19_loss: 0.0193 - dense_20_loss: 0.0206 - dense_21_loss: 0.0179 - dense_22_loss: 0.0165 - dense_23_loss: 0.0208 - dense_24_loss: 0.0179 - dense_25_loss: 0.0209 - dense_26_loss: 0.0181 - dense_27_loss: 0.0199 - dense_28_loss: 0.0192 - dense_29_loss: 0.0219 - dense_30_loss: 0.0209 - dense_31_loss: 0.0191 - dense_32_loss: 0.0191 - dense_33_loss: 0.0183 - dense_34_loss: 0.0186 - dense_35_loss: 0.0218 - dense_36_loss: 0.0168 - dense_37_loss: 0.0213 - dense_38_loss: 0.0188 - dense_39_loss: 0.0186 - dense_40_loss: 0.0198 - dense_41

Pass n° 3 ...
Train on 40000 samples, validate on 40000 samples
Epoch 1/3
40000/40000 [==============================] - 84s 2ms/step - loss: 2.6744 - dense_3_loss: 0.0335 - dense_4_loss: 0.0355 - dense_5_loss: 0.0314 - dense_6_loss: 0.0319 - dense_7_loss: 0.0267 - dense_8_loss: 0.0326 - dense_9_loss: 0.0341 - dense_10_loss: 0.0368 - dense_11_loss: 0.0290 - dense_12_loss: 0.0331 - dense_13_loss: 0.0314 - dense_14_loss: 0.0349 - dense_15_loss: 0.0338 - dense_16_loss: 0.0346 - dense_17_loss: 0.0348 - dense_18_loss: 0.0358 - dense_19_loss: 0.0314 - dense_20_loss: 0.0289 - dense_21_loss: 0.0395 - dense_22_loss: 0.0395 - dense_23_loss: 0.0314 - dense_24_loss: 0.0342 - dense_25_loss: 0.0341 - dense_26_loss: 0.0352 - dense_27_loss: 0.0287 - dense_28_loss: 0.0366 - dense_29_loss: 0.0338 - dense_30_loss: 0.0335 - dense_31_loss: 0.0293 - dense_32_loss: 0.0350 - dense_33_loss: 0.0316 - dense_34_loss: 0.0306 - dense_35_loss: 0.0279 - dense_36_loss: 0.0335 - dense_37_loss: 0.0313 - dense_38_loss: 0

Epoch 2/3
40000/40000 [==============================] - 84s 2ms/step - loss: 1.7235 - dense_3_loss: 0.0212 - dense_4_loss: 0.0243 - dense_5_loss: 0.0226 - dense_6_loss: 0.0195 - dense_7_loss: 0.0237 - dense_8_loss: 0.0193 - dense_9_loss: 0.0198 - dense_10_loss: 0.0183 - dense_11_loss: 0.0244 - dense_12_loss: 0.0199 - dense_13_loss: 0.0200 - dense_14_loss: 0.0188 - dense_15_loss: 0.0188 - dense_16_loss: 0.0213 - dense_17_loss: 0.0242 - dense_18_loss: 0.0232 - dense_19_loss: 0.0212 - dense_20_loss: 0.0209 - dense_21_loss: 0.0239 - dense_22_loss: 0.0239 - dense_23_loss: 0.0225 - dense_24_loss: 0.0247 - dense_25_loss: 0.0223 - dense_26_loss: 0.0199 - dense_27_loss: 0.0194 - dense_28_loss: 0.0235 - dense_29_loss: 0.0219 - dense_30_loss: 0.0239 - dense_31_loss: 0.0219 - dense_32_loss: 0.0224 - dense_33_loss: 0.0209 - dense_34_loss: 0.0198 - dense_35_loss: 0.0176 - dense_36_loss: 0.0221 - dense_37_loss: 0.0175 - dense_38_loss: 0.0188 - dense_39_loss: 0.0206 - dense_40_loss: 0.0196 - dense_41

...

Epoch 3/10
40000/40000 [==============================] - 84s 2ms/step - loss: 23.6266 - dense_3_loss: 0.2959 - dense_4_loss: 0.2835 - dense_5_loss: 0.2885 - dense_6_loss: 0.2912 - dense_7_loss: 0.2892 - dense_8_loss: 0.2911 - dense_9_loss: 0.2868 - dense_10_loss: 0.2851 - dense_11_loss: 0.2912 - dense_12_loss: 0.2895 - dense_13_loss: 0.2914 - dense_14_loss: 0.2958 - dense_15_loss: 0.2889 - dense_16_loss: 0.2848 - dense_17_loss: 0.2933 - dense_18_loss: 0.2938 - dense_19_loss: 0.2947 - dense_20_loss: 0.2892 - dense_21_loss: 0.2846 - dense_22_loss: 0.2932 - dense_23_loss: 0.2909 - dense_24_loss: 0.2874 - dense_25_loss: 0.2874 - dense_26_loss: 0.2882 - dense_27_loss: 0.2867 - dense_28_loss: 0.2890 - dense_29_loss: 0.2863 - dense_30_loss: 0.2930 - dense_31_loss: 0.2924 - dense_32_loss: 0.2945 - dense_33_loss: 0.2897 - dense_34_loss: 0.2960 - dense_35_loss: 0.2959 - dense_36_loss: 0.2908 - dense_37_loss: 0.2941 - dense_38_loss: 0.2950 - dense_39_loss: 0.2903 - dense_40_loss: 0.2862 - dense_

Epoch 00003: early stopping
Pass n° 17 ...
Train on 40000 samples, validate on 40000 samples
Epoch 1/10
40000/40000 [==============================] - 84s 2ms/step - loss: 32.6264 - dense_3_loss: 0.4024 - dense_4_loss: 0.3975 - dense_5_loss: 0.4044 - dense_6_loss: 0.3985 - dense_7_loss: 0.3994 - dense_8_loss: 0.3950 - dense_9_loss: 0.3982 - dense_10_loss: 0.3980 - dense_11_loss: 0.4010 - dense_12_loss: 0.4087 - dense_13_loss: 0.4049 - dense_14_loss: 0.4041 - dense_15_loss: 0.3997 - dense_16_loss: 0.4002 - dense_17_loss: 0.4002 - dense_18_loss: 0.3988 - dense_19_loss: 0.4003 - dense_20_loss: 0.4007 - dense_21_loss: 0.4046 - dense_22_loss: 0.4004 - dense_23_loss: 0.3999 - dense_24_loss: 0.4028 - dense_25_loss: 0.4090 - dense_26_loss: 0.3991 - dense_27_loss: 0.4054 - dense_28_loss: 0.4040 - dense_29_loss: 0.4095 - dense_30_loss: 0.4022 - dense_31_loss: 0.4094 - dense_32_loss: 0.4007 - dense_33_loss: 0.4026 - dense_34_loss: 0.4017 - dense_35_loss: 0.3994 - dense_36_loss: 0.4029 - dense_37_

Epoch 2/10
40000/40000 [==============================] - 84s 2ms/step - loss: 29.8102 - dense_3_loss: 0.3711 - dense_4_loss: 0.3641 - dense_5_loss: 0.3701 - dense_6_loss: 0.3654 - dense_7_loss: 0.3650 - dense_8_loss: 0.3604 - dense_9_loss: 0.3643 - dense_10_loss: 0.3639 - dense_11_loss: 0.3666 - dense_12_loss: 0.3734 - dense_13_loss: 0.3705 - dense_14_loss: 0.3673 - dense_15_loss: 0.3652 - dense_16_loss: 0.3650 - dense_17_loss: 0.3658 - dense_18_loss: 0.3660 - dense_19_loss: 0.3669 - dense_20_loss: 0.3676 - dense_21_loss: 0.3715 - dense_22_loss: 0.3651 - dense_23_loss: 0.3671 - dense_24_loss: 0.3657 - dense_25_loss: 0.3682 - dense_26_loss: 0.3617 - dense_27_loss: 0.3696 - dense_28_loss: 0.3688 - dense_29_loss: 0.3734 - dense_30_loss: 0.3702 - dense_31_loss: 0.3763 - dense_32_loss: 0.3676 - dense_33_loss: 0.3667 - dense_34_loss: 0.3670 - dense_35_loss: 0.3647 - dense_36_loss: 0.3684 - dense_37_loss: 0.3628 - dense_38_loss: 0.3717 - dense_39_loss: 0.3700 - dense_40_loss: 0.3758 - dense_

Epoch 3/10
40000/40000 [==============================] - 84s 2ms/step - loss: 28.6576 - dense_3_loss: 0.3525 - dense_4_loss: 0.3486 - dense_5_loss: 0.3550 - dense_6_loss: 0.3509 - dense_7_loss: 0.3507 - dense_8_loss: 0.3454 - dense_9_loss: 0.3493 - dense_10_loss: 0.3523 - dense_11_loss: 0.3531 - dense_12_loss: 0.3599 - dense_13_loss: 0.3571 - dense_14_loss: 0.3566 - dense_15_loss: 0.3513 - dense_16_loss: 0.3487 - dense_17_loss: 0.3523 - dense_18_loss: 0.3536 - dense_19_loss: 0.3538 - dense_20_loss: 0.3541 - dense_21_loss: 0.3559 - dense_22_loss: 0.3551 - dense_23_loss: 0.3515 - dense_24_loss: 0.3512 - dense_25_loss: 0.3538 - dense_26_loss: 0.3495 - dense_27_loss: 0.3572 - dense_28_loss: 0.3524 - dense_29_loss: 0.3588 - dense_30_loss: 0.3531 - dense_31_loss: 0.3617 - dense_32_loss: 0.3545 - dense_33_loss: 0.3529 - dense_34_loss: 0.3507 - dense_35_loss: 0.3553 - dense_36_loss: 0.3548 - dense_37_loss: 0.3488 - dense_38_loss: 0.3568 - dense_39_loss: 0.3570 - dense_40_loss: 0.3594 - dense_

Epoch 00003: early stopping


### Evaluate solver

In [7]:
quizzes = Xtest.argmax(3)  # quizzes in the (?, 9, 9) shape. From the test set
true_grids = ytest.argmax(3) + 1  # true solutions dont forget to add 1 
smart_guesses = batch_smart_solve(quizzes, solver)  # make smart guesses !

deltas = diff(true_grids, smart_guesses)  # get number of errors on each quizz
accuracy = (deltas == 0).mean()  # portion of correct solved quizzes

In [8]:
print(
"""
Grid solved:\t {}
Correct ones:\t {}
Accuracy:\t {}
""".format(
deltas.shape[0], (deltas==0).sum(), accuracy
)
)


Grid solved:	 10000
Correct ones:	 9933
Accuracy:	 0.9933



Thanks for reading me :)